In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
import scipy.stats as stats

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

In [2]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [3]:
%matplotlib widget

In [4]:
%reload_ext autoreload
%autoreload 2

### Setting up

In [5]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [6]:
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [7]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [13]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)
gfw.scale_Z2 *= -1

***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  175.24060921459508
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  392.683293718069
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  233.0708308046279
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  341.81841601739336
***************************************************
Z2-amplitude:  2.81e+02
reduced chi2:  320.1151243975466
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  235.238026011364
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  250.1490478568309
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  270.7275591713835
***************************************************
Z2-amplitude:  2.67e+02
reduced chi2:  318.6342737392779
*******************

In [14]:
gfw.scale_Z2

1.8474326786101574e+17

### Signal model parameters

In [15]:
def get_Yoffset(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return (437.3 - 25 * 9.5) - (aux.voltage_to_position(np.mean(bb.cant_pos[1])))
def get_Stroke(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return -np.std(aux.voltage_to_position(bb.cant_pos[1]))*np.sqrt(2)*2
def get_Dist(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return 392.2 - aux.voltage_to_position(np.mean(bb.cant_pos[0]))

In [16]:
z_sep = np.mean(height_cal)
y_offset = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Yoffset)(i) for i in tqdm(range(0, 10000, 100)))))
stroke = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Stroke)(i) for i in tqdm(range(0, 10000, 100)))))
dist = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100)))))
G_lambda = 10
print(z_sep, y_offset, stroke, dist, G_lambda)

100%|██████████| 100/100 [00:00<00:00, 696.44it/s]


-15.859100393 4.88214369679 -202.765748131 13.9887662292 10


### Load data

In [111]:
def func_filter(i):
#     ii = index[i]
    ii = i
#     fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/chas_tests/77/Noise_batch_'+str(ii)+'.h5'
#     fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/Noise_add_3_'+str(ii)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(ii)+'.h5'
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking373/Shaking4_'+str(i)+'.h5'
#     fname = r'/data/new_trap_processed/mockfiles/20200320/output/noise/nadav_test/0/Noise_batch_'+str(ii)+'.h5'

    bb = BeadDataFile.BeadDataFile(fname=fname)
    
    res = bb.z2
    return (res)

In [112]:
test_phases = np.array(Parallel(n_jobs=46)(delayed(func_filter)(i) for i in tqdm(range(5000))))

100%|██████████| 5000/5000 [00:54<00:00, 90.95it/s] 


In [113]:
ll = likelihood_analyser.LikelihoodAnalyser()

def calc_bdf3(i):

    data_z2 = np.mean(test_phases[i:(i+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, np.pi],
              'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    noises_std = np.array([1])
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                          signal_freqs=gravity_freqs, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
#     PL_array = A_array*0
#     if (mm_tmp.values[0]>A_array[0])&(mm_tmp.values[0]<A_array[-1]):
#         PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
#         PL_array -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PL_array))
        
    return mm_tmp.values[0]

In [257]:
# likelihood calculator stuff
# gravity_freqs_all = 3*np.arange(1,14)[2:]
# gravity_freqs_all = np.array([30, 33, 36, 39])
# gravity_freqs_all = np.array([3,9,15,21,27,33,39])
gravity_freqs_all = np.array([3,21,15,6])
# gravity_freqs_all = np.array([12,18,24,30])
# gravity_freqs_all = np.array([6,12,18,30,36])
G_lambda = 10

test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])
A_array = np.arange(-10,10,0.1)

fft_angles = np.angle(np.fft.rfft(shaking378_template))
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

MLEs = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
#     phase_all = (0+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    gravity_phases = np.array([phase_all])
    gravity_freqs = np.array([drive_freq])
    gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs, gfw.tf_freq, gfw.tf_ffts[2])
    print('\n********************************************')
    print(gravity_freqs, gravity_phases, gravity_scales)
    print('********************************************\n')
    MLEs.append(Parallel(n_jobs=46)(delayed(calc_bdf3)(i) for i in tqdm(range(5000))))

Loading Gravity Data... 




  0%|          | 0/5000 [00:00<?, ?it/s]

Done!
81 9.74070978211e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data

********************************************
[3] [[ 4.36688428]] [  4.16404109e+16]
********************************************



Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/synchronize.py", line 96, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/synchronize.py", line 96, in _cleanup
    sem_unlink(name)
FileNotFoundError: [Errno 2] No such file or directory
Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = 

 83%|████████▎ | 4140/5000 [00:16<00:02, 324.07it/s]


 84%|████████▎ | 4186/5000 [00:17<00:02, 315.67it/s]


 85%|████████▍ | 4232/5000 [00:17<00:02, 322.79it/s]


 86%|████████▌ | 4278/5000 [00:17<00:02, 315.33it/s]


 86%|████████▋ | 4324/5000 [00:17<00:02, 313.93it/s]


 87%|████████▋ | 4370/5000 [00:17<00:01, 317.07it/s]


 88%|████████▊ | 4416/5000 [00:17<00:01, 322.45it/s]


 89%|████████▉ | 4462/5000 [00:18<00:01, 303.32it/s]


 90%|█████████ | 4508/5000 [00:18<00:01, 313.31it/s]


 91%|█████████ | 4554/5000 [00:18<00:01, 327.54it/s]


 92%|█████████▏| 4600/5000 [00:18<00:01, 318.49it/s]


 93%|█████████▎| 4646/5000 [00:18<00:01, 331.48it/s]


 94%|█████████▍| 4692/5000 [00:18<00:00, 333.60it/s]


 95%|█████████▍| 4738/5000 [00:18<00:00, 327.61it/s]


 96%|█████████▌| 4784/5000 [00:19<00:00, 313.69it/s]


 97%|█████████▋| 4830/5000 [00:19<00:00, 334.57it/s]


 98%|█████████▊| 4876/5000 [00:19<00:00, 335.97it/s]


 98%|█████████▊| 4922/5000 [00:19<00:00, 312.29it/s]


 99%|█████

...




  2%|▏         | 92/5000 [00:00<00:28, 173.26it/s]


  3%|▎         | 138/5000 [00:00<00:26, 184.07it/s]


  4%|▎         | 184/5000 [00:00<00:23, 204.78it/s]


  5%|▍         | 230/5000 [00:01<00:21, 221.93it/s]


  6%|▌         | 276/5000 [00:01<00:20, 232.89it/s]


  6%|▋         | 322/5000 [00:01<00:19, 242.41it/s]


  7%|▋         | 368/5000 [00:01<00:18, 250.68it/s]


  8%|▊         | 414/5000 [00:01<00:18, 251.48it/s]


  9%|▉         | 460/5000 [00:01<00:17, 258.88it/s]


 10%|█         | 506/5000 [00:02<00:16, 269.44it/s]


 11%|█         | 552/5000 [00:02<00:17, 254.82it/s]


 12%|█▏        | 598/5000 [00:02<00:17, 249.26it/s]


 13%|█▎        | 644/5000 [00:02<00:17, 253.47it/s]


 14%|█▍        | 690/5000 [00:02<00:16, 261.12it/s]


 15%|█▍        | 736/5000 [00:03<00:16, 261.44it/s]


 16%|█▌        | 782/5000 [00:03<00:16, 260.57it/s]


 17%|█▋        | 828/5000 [00:03<00:16, 259.82it/s]


 17%|█▋        | 874/5000 [00:03<00:15, 261.61it/s]


 18%|█▊        | 920/5000 


********************************************
[15] [[ 0.30027317]] [  1.05835254e+17]
********************************************






  2%|▏         | 92/5000 [00:00<00:32, 149.14it/s]


  3%|▎         | 138/5000 [00:00<00:27, 173.79it/s]


  4%|▎         | 184/5000 [00:00<00:24, 194.93it/s]


  5%|▍         | 230/5000 [00:01<00:22, 214.35it/s]


  6%|▌         | 276/5000 [00:01<00:20, 227.35it/s]


  6%|▋         | 322/5000 [00:01<00:18, 248.25it/s]


  7%|▋         | 368/5000 [00:01<00:18, 256.34it/s]


  8%|▊         | 414/5000 [00:01<00:17, 267.79it/s]


  9%|▉         | 460/5000 [00:01<00:17, 262.91it/s]


 10%|█         | 506/5000 [00:02<00:16, 270.00it/s]


 11%|█         | 552/5000 [00:02<00:16, 269.15it/s]


 12%|█▏        | 598/5000 [00:02<00:15, 278.57it/s]


 13%|█▎        | 644/5000 [00:02<00:14, 295.26it/s]


 14%|█▍        | 690/5000 [00:02<00:14, 295.88it/s]


 15%|█▍        | 736/5000 [00:02<00:14, 288.83it/s]


 16%|█▌        | 782/5000 [00:03<00:14, 285.94it/s]


 17%|█▋        | 828/5000 [00:03<00:14, 294.29it/s]


 17%|█▋        | 874/5000 [00:03<00:14, 292.86it/s]


 18%|█▊        | 920/5000 


********************************************
[6] [[ 5.68387432]] [  7.38900451e+16]
********************************************






  2%|▏         | 92/5000 [00:00<00:32, 150.01it/s]


  3%|▎         | 138/5000 [00:00<00:27, 178.03it/s]


  4%|▎         | 184/5000 [00:00<00:23, 204.62it/s]


  5%|▍         | 230/5000 [00:01<00:20, 227.88it/s]


  6%|▌         | 276/5000 [00:01<00:19, 244.41it/s]


  6%|▋         | 322/5000 [00:01<00:17, 267.58it/s]


  7%|▋         | 368/5000 [00:01<00:16, 280.22it/s]


  8%|▊         | 414/5000 [00:01<00:15, 300.64it/s]


  9%|▉         | 460/5000 [00:01<00:14, 318.32it/s]


 10%|█         | 506/5000 [00:01<00:14, 302.36it/s]


 11%|█         | 552/5000 [00:02<00:13, 323.40it/s]


 12%|█▏        | 598/5000 [00:02<00:14, 302.70it/s]


 13%|█▎        | 644/5000 [00:02<00:14, 309.12it/s]


 14%|█▍        | 690/5000 [00:02<00:13, 312.23it/s]


 15%|█▍        | 736/5000 [00:02<00:13, 324.47it/s]


 16%|█▌        | 782/5000 [00:02<00:13, 308.52it/s]


 17%|█▋        | 828/5000 [00:02<00:13, 310.41it/s]


 17%|█▋        | 874/5000 [00:03<00:12, 327.25it/s]


 18%|█▊        | 920/5000 

In [ ]:
downsample = 50
alphas_mean = []
alphas_std = []
for data in MLEs:
    
    alphas_mean.append([np.mean(data[i*downsample:(i+1)*downsample]) for i in range(int(10000/downsample))])
    alphas_std.append([np.std(data[i*downsample:(i+1)*downsample])/np.sqrt(downsample) for i in range(int(10000/downsample))])


ERROR:concurrent.futures:exception calling callback for <Future at 0x7fa23f581400 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 340, in __call__
    self.parallel.dispatch_next()
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 769, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 754, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/_para

In [ ]:
gravity_freqs_all

ERROR:concurrent.futures:exception calling callback for <Future at 0x7fa23f581470 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 340, in __call__
    self.parallel.dispatch_next()
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 769, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 754, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/_para

In [ ]:
np.mean(MLEs, axis=1), np.std(MLEs, axis=1)/np.sqrt(5000)

ERROR:concurrent.futures:exception calling callback for <Future at 0x7fa23f581630 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 340, in __call__
    self.parallel.dispatch_next()
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 769, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 754, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/_para

In [ ]:
_,ax = plt.subplots(figsize=(9.2,4))
# [ax.scatter(range(1000), MLEs[i], label=str(gravity_freqs_all[i])) for i in range(5)]
[ax.errorbar(range(len(alphas_mean[0])), alphas_mean[i], yerr=alphas_std[i], fmt='o', label=str(gravity_freqs_all[i])) for i in [0,1]]
ax.legend()

ERROR:concurrent.futures:exception calling callback for <Future at 0x7fa23f581710 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 340, in __call__
    self.parallel.dispatch_next()
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 769, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/parallel.py", line 754, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/home/analysis_user/.local/lib/python3.6/site-packages/joblib/_para

In [255]:
_,ax = plt.subplots()
ax.scatter(alphas_mean[0], alphas_mean[1])
ax.plot(range(-20,20), range(-20,20), 'r--')
ax.set(xlim=(-20,20), ylim=(-20,20))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(-20.0, 20.0), (-20.0, 20.0)]